## CS152 - Assignment 1

The code is this notebook applies the A* search algorithm with various admissible heuristics. 

In [1]:
import numpy as np
import random
import copy
import heapq

In [2]:
class PuzzleNode():
    
    '''
    This is a node class. It captures the state of
    the 8-puzzle and its generalizations, as well as the elements
    needed to implement an A∗ search. It also enables the user to
    print the board state, as well as to test whether the
    puzzle is in its solved state.
    '''
    
    def __init__(self,n=0,g=0,f=0,parent=None,state=None, pruned=False): 
        self.g = g # distance between current node and start node
        self.f = f # total cost of the node, i.e. g + heuristic value
        self.parent = parent # pointer to parent node
        self.pruned = pruned # indication that a node has been pruned
        
        # Generating a board, filled with numbers in a random order and
        # shaping it into the correct board dimentions.
        if state is None:
            self.state = np.array(random.sample(range(self.n**2),
                                                self.n**2)).reshape(self.n, self.n)
        else:
            self.state = np.array(state)
            
        self.n = len(self.state) # size of grid dimension 
    
    def __lt__(self, other_node):     
        '''
        A method that compares one node to another on the basis of their
        f values. Used to order nodes in the heapq data structure that is
        used to construct the frontier. 
        '''
        return (self.f < other_node.f)
    
    def __str__(self):     
        '''A method that prints out an n-by-n grid, showing the board state.'''
        return print(*self.state, sep = ', ')
    
    def solved_test(self):
        '''A method for checking whether the puzzle is in a solved state.'''
        return (np.array_equal(self.state,
                               np.arange(self.n**2).reshape(self.n, self.n)))

### Heuristic functions

In [3]:
# Number of misplaced tiles
def h1(state):
    n = len(state)
    state = np.array(state)
    goal_state= np.arange(n**2).reshape(n,n)
    truth_vals = (state == goal_state)
    summ = sum([1 for x in truth_vals for y in x if not y])
    empty_spot = np.where(state==0)
    if empty_spot != (0,0):
        summ = summ -1
    return(summ)
    
# Manhattan distance
def mhd(state,goal_state,i): 
    state = np.array(state)
    x1, y1 = (np.where(state==i))
    if state[x1[0]][y1[0]]==0:
        return(0)
    else:
        x2, y2 = (np.where(goal_state==i))
        return(abs(x1-x2)+abs(y1-y2))

def h2(state):
    n = len(state)
    goal_state = np.arange(n**2).reshape(n,n)
    # Computes the sum for all tiles in the board
    return(int(sum([mhd(state,goal_state,i) for sublist in state for i in sublist])))

heuristics = [h1, h2]

## Extensions

### 1. Checking for configuration insolvability

In [4]:
def inversion_count(state):
    '''
    Returns the number of possible inversions in a state.
    An inversion is possible when the value of tile i exceeds
    the value of tile j, and i comes before j. The empty tile
    is exluded from consideration. If the count is odd, the state
    cannot be solved.

    '''
    
    state = np.array(state)
    state = state[state!= 0] # deleting the zero
    state = tuple(state.flatten()) # flattening for convenience
    counter = 0
    
    for i in range(len(state)-1):
        for j in range((i+1),(len(state))):
            if state[i] > state[j]:
                counter +=1
    return(counter)

### 2. Disjoint pattern database heuristic

Admissable heuristics are found by relaxing the problems. E.g. in Manhattan distance, we imagine that there are no other tiles in the way of a tile's movement. However, the biggest drawback of both heuristics, used above, is that they do not take into account the interactions between tiles. Therefore, a more helpful heuristic should account for such interactions. 

One way to do so is to split the goal state into subgoals. We can then compute the costs of solving each individual subgoal independently. If such subgoals are disjoint, we can use the sum of costs as a new (and a better) admissible heuristic (Korf & Felner, n.d.). Manhattan distance is a special case of this idea where each set is a single tile. In order to precompute and store such costs, we can use the concept of memoization, i.e. the ability of computers to store the results of expensive function calls and to return the cached results when the same inputs occur again (Wikipedia, n.d.). 

## A* Implementation

In [5]:
def solvePuzzle(init_state, heuristic=None):
    
    '''
    
    init_state - the starting (scrambled) state of the puzzle,
    provided as a list of lists, with the blank space
    represented by the number 0. 
    
    heuristic - a handle to a heuristic function.

    '''
    
    n = len(init_state)
    exp = 0 
    frontier_size = []
    err = 0
    
    # Validation of input arguments
    
    row_size = True # indicator for whether all sublists are of the right size
    for row in init_state:
        if len(row) != n:
            row_size = False
            
    if ((n<=0) or (type(n)!=int) or (sorted([i for row in init_state for i in row]) != list(range(n**2))) or row_size!=True):
#         print('Invalid input, please adjust.') 
        err = -1
        return(0,0,0,None,err)
    
    # Checking for configuration insolvability
    if n%2: # if even-sized board
        if (inversion_count(init_state)%2):
            err = -2
            return(0,0,0,None,err)
    else: # if odd-sized board
        if not (inversion_count(init_state)%2): 
            err = -2
            return(0,0,0,None,err)
    
    # Since the current node is the start node, so the distance betwen them is 0,
    # we can just pass the heuristic value as the f value.
    init_node = PuzzleNode(f=heuristic(init_state), state=init_state)
    open_list = [] # Frontier of nodes to be evaluated
    heapq.heappush(open_list,init_node)
    frontier_size.append(len(open_list))
    
    # Keeping a dictionary of visited nodes (of PuzzleNode class), using their
    # state as a key. Could not keep the state in numpy array form, since
    # it is unhashable; storing in a flat state.
    closed_list = {}
    closed_list[tuple(init_node.state.flatten())] = init_node
    
    while open_list:

        # Gives the node with the least f val,
        # as specified by the comparison mehtod.
        current_node = heapq.heappop(open_list)
        
        if current_node.solved_test():
            node = current_node
            opt_path = [(node.state).tolist()]
            
            while node != init_node:
                opt_path.append((node.parent.state).tolist()) #.tolist()
                node = node.parent 
            steps = len(opt_path)-1
            break
            
        if current_node.pruned == True:
            continue
        
        # Generate a list of possible moves, expressed as
        # additions/subtractions along each axis in matrix.
        
        potential_moves = [[0,-1],[0,1],[-1,0],[1,0]]
        
        empty_spot = np.where(current_node.state==0)
        
        # Neighbors of the empty spot that can be moved in its place
        neighbor_indices = [] 
        
        for sublist in potential_moves:
            new_pos = []
            for i in range(len(sublist)):
                new_pos.append(sublist[i]+int(empty_spot[i]))
            # only keep the ones that satisfy bounds
            if ((0<=new_pos[0] and new_pos[0]<=(n-1)) and (0<=new_pos[1] and new_pos[1]<=(n-1))):
                neighbor_indices.append(new_pos)

        possible_children_nodes = []
        
        for coordinates in neighbor_indices:
            new_current_node = copy.deepcopy(current_node)
            new_current_node.state[coordinates[0],coordinates[1]], new_current_node.state[empty_spot] = 0, new_current_node.state[coordinates[0],coordinates[1]]
            possible_children_nodes.append(new_current_node)

        for child_node in possible_children_nodes: 
            g = current_node.g + 1 # all children are one step away from the parrent (current) node
            
            if tuple(child_node.state.flatten()) in closed_list:
                if (closed_list[tuple(child_node.state.flatten())]).g > g:
                    (closed_list[tuple(child_node.state.flatten())]).pruned = True
                else:
                    # Since the existing identical child in closed_list has a lower cost
                    # we can skip this child and not add it to the open_list.
                    continue 
                    
            # Set the heuristic value from this child node to goal state.
            h = heuristic(child_node.state)
            
            # Assign the f and g values to the child_node.
            child_node.g, child_node.f, child_node.parent = g, (g+int(h)), current_node
            
            # Add new child to the open_list to be explored.
            heapq.heappush(open_list, child_node)
            frontier_size.append(len(open_list))
            
            # Add the node to closed_list.
            closed_list[tuple(child_node.state.flatten())] = child_node
        
            exp +=1
    
    opt_path.reverse()
    
    return(steps, exp, max(frontier_size), opt_path, err)

### Basic Functionality Tests

In [6]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [7]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]],
                                   [[7,0,8],[4,6,1],[5,3,2]],
                                   [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])

In [8]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)

In [9]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])

# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)

# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

In [10]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

### Personal tests

In [11]:
states = [[[7,2,4],[5,0,6],[8,3,1]],
        [[0,2,3],[6,7,5],[8,4,1]],
        [[1,8,0],[5,7,6],[3,4,2]]]

heuristics = {'Misplaced tiles':h1,'Mahattan distance':h2}

for state in states:
    print('State to be solved: ', state)
    
    for heuristic in heuristics:
        steps, exp, max_frontier, opt_path, err = solvePuzzle(state, heuristics[heuristic])
        if exp==0 or max_frontier==0:
            print('This state is unsolvable.')
            break
        else:
            print('Using heuristic: ', heuristic)
            print('Number of steps in optimal solution: ', steps)
            print('Expansion factor: ', exp)
            print('Maximum frontier: ', max_frontier)
    print('\n')

State to be solved:  [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
Using heuristic:  Misplaced tiles
Number of steps in optimal solution:  26
Expansion factor:  55508
Maximum frontier:  16364
Using heuristic:  Mahattan distance
Number of steps in optimal solution:  26
Expansion factor:  5008
Maximum frontier:  1789


State to be solved:  [[0, 2, 3], [6, 7, 5], [8, 4, 1]]
This state is unsolvable.


State to be solved:  [[1, 8, 0], [5, 7, 6], [3, 4, 2]]
Using heuristic:  Misplaced tiles
Number of steps in optimal solution:  22
Expansion factor:  9623
Maximum frontier:  3492
Using heuristic:  Mahattan distance
Number of steps in optimal solution:  22
Expansion factor:  873
Maximum frontier:  327




### References
Korf R.E. & Felner A. (n.d.) https://homepage.iis.sinica.edu.tw/~tshsu/tcg/2012/slides/slide4.pdf

Wikipedia (n.d.) https://en.wikipedia.org/wiki/Memoization